In [17]:
import pandas as pd
import numpy as np
import json
import os
import urllib3
import csv
import logging
import re
import uuid
import shortuuid
from pathlib import Path
from idmc_utils import generate_taskflow, package_import
from urllib.parse import quote

# Set the pandas options
pd.set_option('display.max_columns', None)

# Initialise the job

In [18]:
# Initialise the log file
logging.basicConfig(
    filename='logs/console.log',
    level=logging.DEBUG,
    format='%(asctime)s:%(levelname)s:%(message)s',
)

In [19]:
# Read the config file
logging.info('Reading the config file')
with open('config/config.json', 'r') as infile:
    config = json.load(infile)


# Read the Inputs

In [20]:
# Read the execution plans
logging.info('Reading the execution plans')
dfPlans = pd.read_csv('in/plans.csv', dtype='str', encoding='utf-8', na_filter=False)
dfPlans['plan_step_order'] = dfPlans['plan_step_order'].astype(int)

# Lookup the Converted Mapping Task IDs

In [21]:
# Login to IDMC
logging.info('Logging into IDMC')
http = urllib3.PoolManager()

data = '{ "username": "' + config['idmc']['user'] + '", "password": "' + config['idmc']['password'] + '" }'

url = 'https://' + config['idmc']['host'] + '/saas/public/core/v3/login'
r = http.request(
    'POST', 
    url,
    timeout=3000,
    body=data,
    headers={
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        }
    )
    
# Convert the response into a datframe
result = json.loads(r.data.decode('utf-8'))
sessionID = result['userInfo']['sessionId']


In [22]:
# Get the secure agent ID
logging.info('Getting the secure agent ID')
url = 'https://' + config['idmc']['pod'] + '.' + config['idmc']['host'] + '/saas/api/v2/runtimeEnvironment/name/' + quote(config['idmc']['agentGroupName'])
r = http.request(
    'GET', 
    url,
    timeout=3000,
    headers={
            'Accept': 'application/json',
            'icSessionId': sessionID
        }
    )
    
# Convert the response into a datframe
result = json.loads(r.data.decode('utf-8'))
agentGroupID = result['id']
agentGroupGUID = result['federatedId']
agentGroupName = config['idmc']['agentGroupName']

In [23]:
# Initialise the tasks data frame
logging.info('Getting a list of the mapping tasks')
skip = 0
limit = 200
i = 0
dfTasks = pd.DataFrame()

# Page through mapping task queries
while True:

    # Get a list of the mapping tasks
    url = 'https://' + config['idmc']['pod'] + '.' + config['idmc']['host'] + '/saas/public/core/v3/objects?q=type==%27MTT%27&limit=' + str(limit) + '&skip=' + str(skip)
    r = http.request(
        'GET', 
        url,
        timeout=3000,
        headers={
                'Accept': 'application/json',
                'INFA-SESSION-ID': sessionID
            }
        )
        
    # Convert the response into a datframe
    result = json.loads(r.data.decode('utf-8'))
    dfTmp = pd.json_normalize(result)
    dfResp = dfTmp.copy()
    taskCount = dfResp.iloc[0]['count']
    dfResp = dfResp['objects'].explode()
    dfResp = pd.DataFrame(dfResp)
    dfResp = pd.json_normalize(dfResp['objects'])
    dfTasks = pd.concat([dfTasks, dfResp], ignore_index=True)

    # Break if all records have been returned
    i = i + limit
    if i > taskCount:
        break



In [24]:
# Join the IDMC info onto the plans
dfResp['step_name'] = dfResp['path'].apply(lambda x: os.path.basename(x))
dfResp = dfResp[['step_name','id','path']]
dfResp = dfResp.rename(columns={'id': 'infa_id', 'path': 'infa_path'})
dfPlans = dfPlans.merge(dfResp, how='left', on='step_name')
dfPlans['agent_id'] = agentGroupID
dfPlans['agent_guid'] = agentGroupGUID
dfPlans['agent_name'] = agentGroupName
dfPlans['script_dir'] = config['local']['scriptsDir']
dfPlans['script_args'] = '' # TODO placeholder for any args that need to be passed to the step script

In [25]:
#TODO Remove the below filter when starting to work with non-regular steps
dfPlans = dfPlans[dfPlans['plan_step_type']  == 'REGULAR']

In [26]:
# Generate the unique identifiers
#dfPlans['dac2idmc_step_id'] = dfPlans.apply(lambda x: str(uuid.uuid4()).replace('-',''), axis=1)
dfPlans['dac2idmc_step_id'] = dfPlans.apply(lambda x: "task" + shortuuid.uuid()[:8], axis=1)
#map_order_uuid = { step_order: str(uuid.uuid4()).replace('-','') for step_order in dfPlans['plan_step_order'].unique() }
map_order_uuid = { step_order: "group" + shortuuid.uuid()[:8] for step_order in dfPlans['plan_step_order'].unique() }
dfPlans['dac2idmc_group_id'] = dfPlans['plan_step_order'].map(map_order_uuid)
dfPlans = dfPlans.sort_values('plan_step_order')
dfPlans['dac2idmc_next_id'] = dfPlans['dac2idmc_step_id'].shift(-1)

# Get the next group IDs
dfGroups = dfPlans[['plan_step_order','dac2idmc_group_id']].copy()
dfGroups = dfGroups.drop_duplicates()
dfGroups = dfGroups.sort_values('plan_step_order')
dfGroups['dac2idmc_next_group'] = dfGroups['dac2idmc_group_id'].shift(-1)
dfGroups = dfGroups[['dac2idmc_group_id','dac2idmc_next_group']]
dfPlans = dfPlans.merge(dfGroups, how='inner', on='dac2idmc_group_id')

In [ ]:
dfPlans.head()

,plan_wid,plan_name,plan_inactive_flag,plan_step_wid,plan_step_order,plan_step_type,step_guid,step_wid,step_cmd,step_name,infa_id,infa_path,agent_id,agent_guid,agent_name,script_dir,script_args,dac2idmc_step_id,dac2idmc_group_id,dac2idmc_next_id,dac2idmc_next_group
0,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,4F418CAB76FCD515C5A7F3C94D552CF,0,REGULAR,4F418CAB76FCD515C5A7F3C94D552CF,1911f67f35d9a487283f503fc7ab2ac,SDE_ORA_EmployeeDailySnapshotFact_2,SDE_ORA_EmployeeDailySnapshotFact_2,a0Ks8uNXYKLg38LRNzw6gv,Default/SDE_ORA_EmployeeDailySnapshotFact_2,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,taskXAXKpcQ7,group2bTgyY53,task4kp9qVaR,group335HVMzR
1,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,D1A6418BA9B721A3282ECC5948E8866D,0,REGULAR,,15cf56791e1279889d61b836271199a,SDE_ORA_EmployeeDailySnapshotFact_3,SDE_ORA_EmployeeDailySnapshotFact_3,8gORkCCxfM8hk1VxABRRqd,Default/SDE_ORA_EmployeeDailySnapshotFact_3,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,task4kp9qVaR,group2bTgyY53,taskMtnBnzh4,group335HVMzR
2,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,C4FA37D5638DC49425B9A3E8572EF1,0,REGULAR,,17d561355624ced1967afc8fb7a1836e,SDE_ORA_EmployeeDailySnapshotFact_4,SDE_ORA_EmployeeDailySnapshotFact_4,6vPlMGJ7sw6juVvUtKpxYq,Default/SDE_ORA_EmployeeDailySnapshotFact_4,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,taskMtnBnzh4,group2bTgyY53,task3D8S3X6Q,group335HVMzR
3,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,C1CB82CC89178BE3424CEE1896ED,0,REGULAR,,88818EC577E3BDEA5FAF5D56ED6E8442,,TASK_GROUP_Extract_EmployeeDailySnapshotFact_P...,7PjDO1zxryekytQ7Oh1zEP,Default/TASK_GROUP_Extract_EmployeeDailySnapsh...,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,task3D8S3X6Q,group2bTgyY53,taskkNZLay2f,group335HVMzR
4,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,C9EA5C489D4F8CE8B019A694E906B6B,1,REGULAR,,6bcdbdd6812a9ea517908ca566436fb,SDE_ORA_EmployeeDailySnapshotFact_1,SDE_ORA_EmployeeDailySnapshotFact_1,aeOQ6VugmCxipbYyKlDRUr,Default/SDE_ORA_EmployeeDailySnapshotFact_1,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,taskkNZLay2f,group335HVMzR,taskUQoBqFZ6,group6BVtF5Pw
5,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,743C7FED4E93373BF741E35ACCC2FB,1,REGULAR,,3e589d7525743f1159f9aef6d26dfb6d,SDE_ORA_EmployeeDailySnapshotFact_PerformanceR...,SDE_ORA_EmployeeDailySnapshotFact_PerformanceR...,bspgWgKqNAAkdFCD8X1EGw,Default/SDE_ORA_EmployeeDailySnapshotFact_Perf...,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,taskUQoBqFZ6,group335HVMzR,taskFrpZKQYc,group6BVtF5Pw
6,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,352497EEA2C3B9CC108AA6D1C1D461,2,REGULAR,,cddaf55424eff460187a8840ba81641,SDE_ORA_EmployeeDailySnapshotFact_1_Update,SDE_ORA_EmployeeDailySnapshotFact_1_Update,980nZ1ZyECqlJELIoT6seu,Default/SDE_ORA_EmployeeDailySnapshotFact_1_Up...,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,taskFrpZKQYc,group6BVtF5Pw,task7kXfQ2T8,groupKRUFE4Lx
7,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,453CB6AF9425B92F9830964A1BD6B2F8,3,REGULAR,,8049dfe7391b73f4b03f91685796a525,SDE_ORA_EmployeeDailySnapshotFact,SDE_ORA_EmployeeDailySnapshotFact,48EkOr6iGA3jPFoSNlYhmt,Default/SDE_ORA_EmployeeDailySnapshotFact,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,task7kXfQ2T8,groupKRUFE4Lx,task35yQJTB6,groupN5QMFreF
8,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,9228D527E27845B78E5B48C8AEAE7DFE,4,REGULAR,,66477cb62a1282f18536b1d9f122ee,SIL_EmployeeDailySnapshotFact,SIL_EmployeeDailySnapshotFact,hrqZyoLnJWAbb4e5QoamgT,Default/SIL_EmployeeDailySnap

In [28]:
# Log an error if any plans did not find an existing matching task
dfMissing = dfPlans[(dfPlans['infa_id'] == '') | (dfPlans['infa_id'].isna())].copy()
if len(dfMissing.index) > 0:
    logging.error('Some plans are missing a converted mapping task. Please see "out/missing_tasks.csv" for more details')
    dfMissing.to_csv('out/missing_tasks.csv', index=False, quoting=csv.QUOTE_ALL)

# Generate the Taskflows

In [29]:
#TODO add loop through execution plans
planIds = dfPlans['plan_wid'].unique()

for planId in planIds:

    # Get the plan for the current id
    dfPlan = dfPlans[dfPlans['plan_wid'] == planId].copy()

    # Generate the taskflow ID
    #taskflowID = str(uuid.uuid4()).replace('-','')
    taskflowID = shortuuid.uuid()
    taskflowName = dfPlan.iloc[0]['plan_name']
    taskflowName = re.sub(r'\W+', '_', taskflowName)

    logging.info(f'Create workspace tree "tmp/{ taskflowName }/Explore/Default"')

    # Create the workspace directories
    treePath = Path(f'tmp/{ taskflowName }/Explore/Default')
    treePath.mkdir(parents=True, exist_ok=True)

    logging.info(f'Generating taskflow "{ taskflowName }"')
    generate_taskflow.generate_taskflow(taskflowID, taskflowName, dfPlan)

    logging.info(f'Packaging taskflow "{ taskflowName }"')
    package_import.package_import(taskflowID, taskflowName, dfPlan)

    logging.info(f'Done!')

group335HVMzR == 2
group6BVtF5Pw == 1


# Testing below this line